<a href="https://colab.research.google.com/github/raj-vijay/da/blob/master/01_OR_Tools_Knapsack_Problem_CP_SAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Optimization problems**

<p align = 'justify'>The goal of optimization is to find the best solution to a problem out of a large set of possible solutions.

A typical optimization problem is the logistics shipping problem. 

A shipping company delivers packages to its customers using a fleet of trucks, and the company must assign packages to trucks, and then choose a route for each truck to deliver its packages. Each possible assignment of packages and routes has a cost, based on the total travel distance for the trucks, and possibly other factors as well. The problem is to choose the assignments of packages and routes that has the least cost.

The problem has the following elements:

- **Objective** 

The quantity to be optimized. For the above problem, the objective is to minimize cost. To set up an optimization problem, the value of the objective for any possible solution should be defined, and is called the objective function. 

An optimal solution is one for which the value of the objective function is the best. (i.e. a maximum or a minimum.)

- **Constraints** 

The constrains are restrictions on the set of possible solutions, based on the specific requirements of the problem. For e.g., if the shipping company can't assign packages above a given weight to trucks, this would impose a constraint on the solutions. 

A feasible solution is one that satisfies all the given constraints for the problem, without necessarily being optimal.

The first step in solving an optimization problem is identifying the objective and constraints.</p>

##**Packing**##

<p align = 'justify'>The goal of packing problems is to find the best way to pack a set of items of given sizes into containers with fixed capacities. A typical application is loading boxes onto delivery trucks efficiently. 

It is often not possible to pack all the items, due to the capacity constraints. 

The problem is to find a subset of the items with maximum total size that will fit in the containers.

There are many types of packing problems. Two of the most important are knapsack problems and bin packing.</p>

##**Knapsack problems**

In the simple knapsack problem, there is a single container (a knapsack). The items have values as well as sizes, and the goal is to pack a subset of the items that has maximum total value.

For the special case in which value is equal to size, the goal is to maximize the total size of the packed items.

There are also more general versions of the knapsack problem. 

- **Multidimensional knapsack problem**

Here, the items have more than one physical attribute such as weight and volume, and the knapsack has a capacity for each quantity. Here, the term dimension does not necessarily refer to the usual spatial dimensions of height, length, and width. However, some problems might involve spatial dimensions, for example, finding the optimal way to pack rectangular boxes into a rectangular storage bin.

- **Multiple knapsack problem**

Here, there are multiple knapsacks, and the goal is to maximize the total value of the packed items in all knapsacks.

Note that you can have a multidimensional problem with a single knapsack, or a multiple knapsack problem with just one dimension.

In [ ]:
!pip install ortools

     |████████████████████████████████| 32.7MB 115kB/s 


##**Lab Exercise**

In Lecture 2 we looked at the knapsack problem. The goal is to find the most valuable combination of items to pack without exceeding the weight capacity limit. 

Consider the following specific instance:
- The capacity limit of the knapsack is 15kg
- There are 5 items as depicted in the picture to the right
- Each of the 5 items has a weight and a value printed on the box in the picture to the right


(Remark: The Google OR Tools contain an optimised Knapsack solver for this problem domain. Please do NOT use this dedicated solver for the exercise but use the generic CP-SAT solver instead)

In [ ]:
from ortools.sat.python import cp_model

In [ ]:
def BinpackingProblemSat():
  """Solves a bin-packing problem using the CP-SAT solver."""
  # Data.
  bin_capacity = 100
  slack_capacity = 20
  num_bins = 5
  all_bins = range(num_bins)
  
  items = [(20, 6), (15, 6), (30, 4), (45, 3)]
  num_items = len(items)
  all_items = range(num_items)

  # Model.
  model = cp_model.CpModel()

  # Main variables.
  x = {}
  for i in all_items:
    num_copies = items[i][1]
    for b in all_bins:
      x[(i, b)] = model.NewIntVar(0, num_copies, 'x_%i_%i' % (i, b))

  # Load variables.
  load = [model.NewIntVar(0, bin_capacity, 'load_%i' % b) for b in all_bins]
  
  # Slack variables.
  slacks = [model.NewBoolVar('slack_%i' % b) for b in all_bins]
  
  # Links load and x.
  for b in all_bins:
    model.Add(load[b] == sum(x[(i, b)] * items[i][0] for i in all_items))
    
    # Place all items.
    for i in all_items:
      model.Add(sum(x[(i, b)] for b in all_bins) == items[i][1])
      
      # Links load and slack through an equivalence relation.
      safe_capacity = bin_capacity - slack_capacity
      for b in all_bins:
        # slack[b] => load[b] <= safe_capacity.
        model.Add(load[b] <= safe_capacity).OnlyEnforceIf(slacks[b])
        # not(slack[b]) => load[b] > safe_capacity.
        model.Add(load[b] > safe_capacity).OnlyEnforceIf(slacks[b].Not())

  # Maximize sum of slacks.
  model.Maximize(sum(slacks))

  # Solves and prints out the solution.
  solver = cp_model.CpSolver()
  status = solver.Solve(model)
  print('Solve status: %s' % solver.StatusName(status))
  if status == cp_model.OPTIMAL:
    print('Optimal objective value: %i' % solver.ObjectiveValue())
    print('Statistics')
    print('  - conflicts : %i' % solver.NumConflicts())
    print('  - branches  : %i' % solver.NumBranches())
    print('  - wall time : %f s' % solver.WallTime())

In [ ]:
BinpackingProblemSat()

Solve status: OPTIMAL
Optimal objective value: 1
Statistics
  - conflicts : 63
  - branches  : 98
  - wall time : 0.021351 s
